In [1]:
! pip install transformers
! pip install datasets
!pip install evaluate
!pip install seqeval



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 163 kB 70.0 MB/s 
     |████████████████████████████████| 7.6 MB 71.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 441 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 95 kB 4.8 MB/s 
     |████████████████████████████████| 115 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 82.1 MB/s 
     |████████████████████████████████| 115 kB 61.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully unin

In [2]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModel
from collections import defaultdict
from transformers import get_linear_schedule_with_warmup
import time
import seqeval
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.scheme import IOB2
from seqeval.scheme import IOB1
import evaluate

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# from sklearn.metrics import f1_score
import datasets
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from transformers import BertModel, BertTokenizer
import os
from pandas.core.frame import DataFrame
from copy import deepcopy

In [4]:
from google.colab import drive
drive.mount('/content/drive')
seqeval = evaluate.load('seqeval')


Mounted at /content/drive


In [5]:
# enumerate dataset, each value is a dict data type, each value has to keys which are tokens and ner_tags
# i extract the tokens and ner_tags from dataset and put them in two lists.
def get_fulldata(dataset):
    tokens = []
    ner_tags = []
    for idx,value in enumerate(dataset):
        tokens.append(value['tokens'])
        ner_tags.append(value['ner_tags'])
    return tokens,ner_tags

In [6]:

def produce_database(tokens,ner_tags,num,label_num): 
    # initialize a label_list which includes 1-279 numbers
    label_list = [x for x in range(1,(label_num+1))] 
    tk = []
    nt = []
    index_save = []#save the index to check if they have been chosed
    
    for i in label_list:
        # 计数器tmp
        tmp = 0
        for idx,value in enumerate(ner_tags):
            #if the label i in the value of ner_tags, add it in the tk list
            if i in value:
                # check if the data has been saved in the index_save list by index
                if idx not in index_save:
                    nt.append(value)
                    tk.append(tokens[idx])
                    tmp = tmp+1
                    index_save.append(idx)
            # if temp == 5, means that there are 5 items of data has been saved, then break the for loop
            if tmp==num:
                
                # print('finished label '+str(i))
                
                break
               # if the data is the last one and tmp is still 0, print(no any data), then break, else print(data is less then 5)
            if(idx == (len(ner_tags)-1)):
                if tmp==0:
                    # print('label '+str(i)+' no any data')
                    break
                # else:
                #     # print('label '+str(i)+' less than '+ str(num))
     #convert it to dataframe and save it as datasets              
    df = pd.DataFrame.from_dict({'tokens':tk,'ner_tags':nt})
    final_dataset = datasets.Dataset.from_pandas(df)
    return  final_dataset,df

In [7]:
import re
import random

# mask number function, input a list and the probablity, traverse the token list and detect number by regular expression, 
# and convert it to a [MASK] token, finaaly return the list
def random_mask_number(probability, tokens_list):

    save = []
    processed_text = []
    for i in range(probability):
        save.append(1)
    for x in range(100 - probability):
        save.append(0)
    for token in tokens_list:
      if re.fullmatch(r"(\d+[\d,.]*)|([,.]\d+)", token):
        a = random.choice(save)
        if a == 0:
          processed_text.append(token)
        if a == 1:
          processed_text.append('[MASK]')
      else:
        processed_text.append(token)
    return processed_text

In [8]:
def data_preprocess(data,max_len):
  # max_length = 200, so remove or pad the tokens and ner_tags to 200
  tokens = []
  ner_tags = []
  for i in range(len(data)):
    if(len(data[i]['tokens'])>max_len): 
      tokens.append(data[i]['tokens'][:max_len])
    else:
      tokens.append(data[i]['tokens'])
    if(len(data[i]['ner_tags'])>max_len):
      ner_tags.append(data[i]['ner_tags'][:max_len])
    else:
      ner_tags.append(data[i]['ner_tags'])
  return tokens,ner_tags

In [9]:

class AutoNerClassifier(nn.Module):

  def __init__(self, n_classes,PRE_TRAINED_MODEL_LIST:list,model_code:int):
    super(AutoNerClassifier, self).__init__()
    
    self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_LIST[model_code])

    self.drop = nn.Dropout(p=0.1)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)
  
  def forward(self, input_ids, attention_mask):
    output = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask,
   
    )
    output = self.drop(output[0])
    
    return self.out(output)

In [10]:
def produce_input_data(tokens,ner_tags,tokenizer,mask_number,probability):

  token_id= []
  ner_tags_id = []
  attention_mask = []
  # process data, do mask number, add [cls],[sep] tokens to the start and end of the tokens,
  #produce attention masks
  #produce targets matrix by padding -1
  for i in tokens:
    if(mask_number == True):
      i = random_mask_number(probability, i)
    text = ['[CLS]']+i+['[SEP]']
    token_id.append(tokenizer.convert_tokens_to_ids(text))
    attention_mask.append(np.ones_like(text))
    
  all_pad_tokens = pad_sequences(token_id, maxlen=202, padding='post', truncating='post')
  all_mask = pad_sequences(attention_mask, maxlen=202, padding='post', truncating='post')

  for j in ner_tags:
    tags = [-1]+j+[-1]
    ner_tags_id.append(tags)
  all_pad_tags = pad_sequences(ner_tags_id, maxlen=202, padding='post', truncating='post',value=-1)

  glb_id = []
  for i in range(len(tokens)):
    glb_id.append(i)

  token_len = []
  for i in tokens:
    token_len.append(len(i))
 
  return{
      
      'input_ids': torch.tensor(all_pad_tokens,dtype=torch.long),
      'attention_mask': torch.tensor(all_mask, dtype=torch.long),
      'targets': torch.tensor(all_pad_tags, dtype=torch.long),
      'glb_id': torch.tensor(glb_id, dtype=torch.long),
      'token_len':torch.tensor(token_len, dtype=torch.long),
    }

In [11]:
def convert_BIO2IO(BIO_label,ner_tags):
  
  IO_label = {}
  tmp = []

  for i in BIO_label.keys():
    if BIO_label[i] is 'O':
      tmp.append(BIO_label[i])
    elif BIO_label[i][2:] not in tmp:
      tmp.append(BIO_label[i][2:])

  for i,j in zip(tmp,range(len(tmp))):
    if j == 0:
      IO_label[j] = i
    else:
      IO_label[j] = 'I-{}'.format(i)
  convert_dict = {}

  for i in IO_label.keys():
    for j in all_label_name.keys():
      if IO_label[i] == 'O':
        convert_dict[0] = 0
      if IO_label[i][2:] == BIO_label[j][2:]:
        convert_dict[j] = i
  for index,value in enumerate(ner_tags):
    for idx, val in enumerate(value):
      value[idx] = convert_dict[value[idx]]

  return ner_tags,IO_label

In [12]:

class DataSequence(Dataset):

    def __init__(self, data):

      self.features = data['input_ids']
      self.mask = data['attention_mask']
      self.labels = data['targets']
      self.glb_id = data['glb_id']
      self.token_len = data['token_len']

    def __len__(self):

        return len(self.features)

    def __getitem__(self, index):

        return self.features[index],self.mask[index],self.labels[index],self.glb_id[index],self.token_len[index]

In [19]:
def get_all_data(data,tokenizer,mask_number,max_len,probability,random_seed,shuffle:bool):
  #produce tokens , ner_tags
  tokens,ner_tags = data_preprocess(data,max_len)
  #using data_preprocess function to convert the data to a dict which include attention mask, inputs id, and targets
  data = produce_input_data(tokens,ner_tags,tokenizer,mask_number,probability)
  data = DataSequence(data)
  if shuffle == True:
    dataloader = DataLoader(data, num_workers=10, batch_size=4, shuffle=True, worker_init_fn=np.random.seed(random_seed))
  else:
    dataloader = DataLoader(data, num_workers=10, batch_size=4)

  return dataloader

In [14]:
sec_tokenizer = AutoTokenizer.from_pretrained("nlpaueb/sec-bert-base",do_lower_case=True)
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone',do_lower_case=True)

bert_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case=True)

Downloading:   0%|          | 0.00/263 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/568 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/221k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [15]:
class print_bad_case():
  def __init__(self,all_label_name):
    self.sentences = []
    self.true_label = []
    self.pred_label = []
    self.all_label_name = all_label_name

  def get_batch_badcase(self,glb_id,real_targets,pred_targets,token_lens,tokens):
    for id in glb_id:
      self.sentences.append([])
      for token in tokens[id]:
        self.sentences.append([token])

      
    for real_target,token_len in zip(real_targets,token_lens):
  
      self.true_label.append([])
      for token in real_target[1:token_len+1]:
        self.true_label.append(self.all_label_name[token.item()])

    for pred_target,token_len in zip(pred_targets,token_lens):

      self.pred_label.append([])
      for token in pred_target[1:token_len+1]:
        self.pred_label.append(self.all_label_name[token.item()])

  def generate_badcase_file(self):
    column_sentence = pd.DataFrame({'sentence':self.sentences})
    column_real_label = pd.DataFrame({'real_label':self.true_label})
    column_pred_label = pd.DataFrame({'pred_label':self.pred_label})
    
    df = pd.concat([column_sentence,column_real_label,column_pred_label],axis = 1)
    return df 
  def f1_calculation(self):

    true = [value for value in self.true_label if value != []]
    pred = [value for value in self.pred_label if value != []]

    f1 = f1_score([true], [pred],mode='strict',scheme=IOB2) 
    return f1
   
    

    
    


In [16]:
def train_and_eval(model, 
  train_loader, 
  validation_loader,
  train_tokens,
  validation_tokens,
  loss_fn, 
  optimizer, 
  all_label_name, 
  device,
  epoch,
  label_num,
  need_badcase_file:bool,
  model_name:str,
  database_name:str):
  model.train()
  f1_state = 0
  for epo in range(epoch):
    
    total_loss_train = 0
    total_loss_validation = 0


    # for print bad case using
    train_badcase = print_bad_case(all_label_name)
    validation_badcase = print_bad_case(all_label_name)
    
    
    for data in train_loader:
      
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      targets = data[2].to(device)
      glb_id = data[3].to(device)
      token_lens = data[4].to(device)



      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      
      real_targets = deepcopy(targets.cpu())
      prediction_result, prediction_position = torch.max(outputs, dim=2)
      loss = loss_fn(outputs.view(-1, label_num), targets.view(-1))
       
      #for printing bad case
      train_badcase.get_batch_badcase(glb_id,real_targets,prediction_position,token_lens,train_tokens)


      total_loss_train += loss.item()
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    model.eval()

    with torch.no_grad():
      for data in validation_loader:
    
        input_ids = data[0].to(device)
        attention_mask = data[1].to(device)
        targets = data[2].to(device)
        glb_id = data[3].to(device)
        token_lens = data[4].to(device)

        outputs = model(
          input_ids=input_ids,
          attention_mask=attention_mask
        )

        real_targets = deepcopy(targets.cpu())

        prediction_result, prediction_position = torch.max(outputs, dim=2)
        
        loss = loss_fn(outputs.view(-1, label_num), targets.view(-1))
        total_loss_validation += loss.item()


        # for print bad case
        validation_badcase.get_batch_badcase(glb_id,real_targets,prediction_position,token_lens,validation_tokens)
        
    
    #for print f1 using:
    train_f1 = train_badcase.f1_calculation()
    val_f1 = validation_badcase.f1_calculation()
    
    if(val_f1 > f1_state):
      f1_state = val_f1
      torch.save(model.state_dict(), model_name+'_'+database_name+'_state.bin')

        # for printing bad case using

      # ====================== start ===========================================
      if need_badcase_file == True:

        df_train = train_badcase.generate_badcase_file()
        df_validation = validation_badcase.generate_badcase_file()

      # ====================== end ===========================================

    val_loss = total_loss_validation / len(validation_loader)   
    train_loss = total_loss_train / len(train_loader)
    print('Epoch [{}] Train Loss: {:.4f}, Train F1_SCORE: {:.4f}, Valid Loss: {:.4f}, Valid F1_SCORE: {:.4f},'
                      .format(epo+1,train_loss, train_f1, val_loss, val_f1))
    
    
  if need_badcase_file == True:

    return df_train,df_validation


In [17]:
def test_prediction(model, 
  test_loader, 
  test_tokens,
  all_label_name,
  loss_fn, 
  optimizer,
  device,
  need_badcase_file:bool
  ):
  
  
  test_badcase = print_bad_case(all_label_name)


  model.eval()
  with torch.no_grad():
    for data in test_loader:
      input_ids = data[0].to(device)
      attention_mask = data[1].to(device)
      targets = data[2].to(device)
      glb_id = data[3].to(device)
      token_lens = data[4].to(device)
      outputs = model(
        input_ids=input_ids,
        attention_mask=attention_mask
      )
      prediction_result, prediction_position = torch.max(outputs, dim=2)

      real_targets = deepcopy(targets.cpu())
  

      #for printing bad case
      #====================== start =====================================
      test_badcase.get_batch_badcase(glb_id,real_targets,prediction_position,token_lens,test_tokens)

     
      # ==================================== end =============================
      
      
  test_f1 = test_badcase.f1_calculation()

  print('Test F1_SCORE: {:.4f}'.format(test_f1))
  # for bad case printing
  # ====================== start ===========================================
  
  if need_badcase_file == True:

    df_test = test_badcase.generate_badcase_file()
  # ====================== end ===========================================
  if need_badcase_file == True:
    return df_test

In [18]:
multi_task_dataset = ['Amortization','BusinessAcquisition','BusinessCombination','DebtInstrument']
few_shot_numbers = [1,5,10,20]

In [ ]:

for task_name in multi_task_dataset:
  
  for few_shot_num in few_shot_numbers:
    os.chdir('/content/drive/MyDrive/latest_dataset/{}'.format(task_name))
    #load dataset label
    all_label_name = pd.read_csv('{}.csv'.format(task_name))

    #get dataset label total number
    label_num = len(all_label_name)
    all_label_name = all_label_name.to_dict()
    all_label_name = all_label_name['0']

    #load dataset

    train = datasets.load_from_disk('{}_train_number'.format(task_name))
    test = datasets.load_from_disk('{}_test_number'.format(task_name))
    validation = datasets.load_from_disk('{}_validation_number'.format(task_name))
    
    train,_ = produce_database(train['tokens'],train['ner_tags'],few_shot_num,label_num)

    #  dataloder settings
    mask_number = False
    max_len = 200
    probability = 100
    random_seed = 42

    train_tokens,_ = data_preprocess(train,max_len)
    validation_tokens,_ = data_preprocess(validation,max_len)
    test_tokens,_ = data_preprocess(test,max_len)

    train_dataloader = get_all_data(train,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    validation_dataloader = get_all_data(validation,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)
    test_dataloader = get_all_data(test,finbert_tokenizer,mask_number,max_len,probability,random_seed,shuffle = False)

    # lable annotation


    # training parameters settings
    if few_shot_num == 1:
      EPOCHS = 40
    else:
      EPOCHS = 20
 
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    loss_fn = nn.CrossEntropyLoss(ignore_index=-1).to(device)
    Batch_size = 16
    model_name = 'fin_bert'
    # model selection
    PRE_TRAINED_MODEL_LIST = ['bert-base-uncased','yiyanghkust/finbert-tone',"nlpaueb/sec-bert-shape"]

    model = AutoNerClassifier(label_num,PRE_TRAINED_MODEL_LIST,1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    need_badcase_file = True

    #train


    df_train,df_validation = train_and_eval(model, 
    train_dataloader, 
    validation_dataloader,
    train_tokens,
    validation_tokens,
    loss_fn, 
    optimizer, 
    all_label_name, 
    device,
    EPOCHS,
    label_num,
    need_badcase_file,
    model_name,
    task_name)

    model_path = '{}_{}_state.bin'.format(model_name,task_name) 

    model.load_state_dict(torch.load(model_path, map_location=torch.device("cuda:0")))

    df_test = test_prediction(model, 
      test_dataloader, 
      test_tokens,
      all_label_name,
      loss_fn, 
      optimizer,
      device,
      need_badcase_file)
    
    df_train = df_train.mask(df_train.applymap(str).eq('[]'))
    df_train =df_train.drop([0])
    

    df_validation = df_validation.mask(df_validation.applymap(str).eq('[]'))
    df_validation =df_validation.drop([0])

    df_test = df_test.mask(df_test.applymap(str).eq('[]'))
    df_test =df_test.drop([0])

    if not os.path.exists('/content/drive/MyDrive/finer_badcase_print/{}'.format(task_name)):
      os.mkdir('/content/drive/MyDrive/finer_badcase_print/{}'.format(task_name))
    os.chdir('/content/drive/MyDrive/finer_badcase_print/{}'.format(task_name))
     

    df_train.to_csv('{}_{}_train_bad_case.csv'.format(task_name,few_shot_num),index=False)
    df_validation.to_csv('{}_{}_validation_bad_case.csv'.format(task_name,few_shot_num),index=False)
    df_test.to_csv('{}_{}_test_bad_case.csv'.format(task_name,few_shot_num),index=False)

    

    


Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Loss: 1.8420, Train F1_SCORE: 0.0000, Valid Loss: 1.1584, Valid F1_SCORE: 0.0000,
Epoch [2] Train Loss: 0.8591, Train F1_SCORE: 0.0000, Valid Loss: 0.5478, Valid F1_SCORE: 0.0000,
Epoch [3] Train Loss: 0.3679, Train F1_SCORE: 0.0000, Valid Loss: 0.2884, Valid F1_SCORE: 0.0000,
Epoch [4] Train Loss: 0.2081, Train F1_SCORE: 0.0000, Valid Loss: 0.2236, Valid F1_SCORE: 0.0000,
Epoch [5] Train Loss: 0.1737, Train F1_SCORE: 0.0000, Valid Loss: 0.2132, Valid F1_SCORE: 0.0000,
Epoch [6] Train Loss: 0.1627, Train F1_SCORE: 0.0000, Valid Loss: 0.2061, Valid F1_SCORE: 0.0000,
Epoch [7] Train Loss: 0.1461, Train F1_SCORE: 0.0000, Valid Loss: 0.1936, Valid F1_SCORE: 0.0000,
Epoch [8] Train Loss: 0.1244, Train F1_SCORE: 0.0000, Valid Loss: 0.1792, Valid F1_SCORE: 0.0000,
Epoch [9] Train Loss: 0.1046, Train F1_SCORE: 0.0000, Valid Loss: 0.1655, Valid F1_SCORE: 0.0000,
Epoch [10] Train Loss: 0.0891, Train F1_SCORE: 0.0000, Valid Loss: 0.1530, Valid F1_SCORE: 0.0253,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Loss: 1.1432, Train F1_SCORE: 0.0068, Valid Loss: 0.2423, Valid F1_SCORE: 0.0000,
Epoch [2] Train Loss: 0.2734, Train F1_SCORE: 0.0000, Valid Loss: 0.2109, Valid F1_SCORE: 0.0000,
Epoch [3] Train Loss: 0.2233, Train F1_SCORE: 0.0000, Valid Loss: 0.1527, Valid F1_SCORE: 0.0119,
Epoch [4] Train Loss: 0.1524, Train F1_SCORE: 0.1351, Valid Loss: 0.1248, Valid F1_SCORE: 0.0771,
Epoch [5] Train Loss: 0.1191, Train F1_SCORE: 0.1915, Valid Loss: 0.1100, Valid F1_SCORE: 0.0953,
Epoch [6] Train Loss: 0.0928, Train F1_SCORE: 0.3596, Valid Loss: 0.0964, Valid F1_SCORE: 0.1321,
Epoch [7] Train Loss: 0.0733, Train F1_SCORE: 0.3944, Valid Loss: 0.0918, Valid F1_SCORE: 0.0488,
Epoch [8] Train Loss: 0.0615, Train F1_SCORE: 0.3793, Valid Loss: 0.0857, Valid F1_SCORE: 0.1030,
Epoch [9] Train Loss: 0.0508, Train F1_SCORE: 0.5882, Valid Loss: 0.0801, Valid F1_SCORE: 0.2920,
Epoch [10] Train Loss: 0.0411, Train F1_SCORE: 0.7000, Valid Loss: 0.0772, Valid F1_SCORE: 0.4148,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Loss: 0.4196, Train F1_SCORE: 0.0049, Valid Loss: 0.1194, Valid F1_SCORE: 0.0000,
Epoch [2] Train Loss: 0.1433, Train F1_SCORE: 0.2634, Valid Loss: 0.0894, Valid F1_SCORE: 0.2716,
Epoch [3] Train Loss: 0.1008, Train F1_SCORE: 0.6014, Valid Loss: 0.0805, Valid F1_SCORE: 0.5387,
Epoch [4] Train Loss: 0.0841, Train F1_SCORE: 0.6899, Valid Loss: 0.0763, Valid F1_SCORE: 0.4834,
Epoch [5] Train Loss: 0.0712, Train F1_SCORE: 0.7339, Valid Loss: 0.0653, Valid F1_SCORE: 0.5705,
Epoch [6] Train Loss: 0.0502, Train F1_SCORE: 0.8023, Valid Loss: 0.0553, Valid F1_SCORE: 0.7275,
Epoch [7] Train Loss: 0.0308, Train F1_SCORE: 0.8889, Valid Loss: 0.0536, Valid F1_SCORE: 0.7500,
Epoch [8] Train Loss: 0.0242, Train F1_SCORE: 0.9032, Valid Loss: 0.0566, Valid F1_SCORE: 0.7669,
Epoch [9] Train Loss: 0.0197, Train F1_SCORE: 0.9529, Valid Loss: 0.0561, Valid F1_SCORE: 0.7405,
Epoch [10] Train Loss: 0.0174, Train F1_SCORE: 0.9834, Valid Loss: 0.0549, Valid F1_SCORE: 0.7639,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch [1] Train Loss: 2.7652, Train F1_SCORE: 0.0000, Valid Loss: 1.4607, Valid F1_SCORE: 0.0031,
Epoch [2] Train Loss: 0.8282, Train F1_SCORE: 0.0000, Valid Loss: 0.3691, Valid F1_SCORE: 0.0000,
Epoch [3] Train Loss: 0.3435, Train F1_SCORE: 0.0000, Valid Loss: 0.2676, Valid F1_SCORE: 0.0000,
Epoch [4] Train Loss: 0.3421, Train F1_SCORE: 0.0000, Valid Loss: 0.2767, Valid F1_SCORE: 0.0000,
Epoch [5] Train Loss: 0.3400, Train F1_SCORE: 0.0000, Valid Loss: 0.2638, Valid F1_SCORE: 0.0000,
Epoch [6] Train Loss: 0.3076, Train F1_SCORE: 0.0000, Valid Loss: 0.2372, Valid F1_SCORE: 0.0000,
Epoch [7] Train Loss: 0.2673, Train F1_SCORE: 0.0000, Valid Loss: 0.2218, Valid F1_SCORE: 0.0000,
Epoch [8] Train Loss: 0.2450, Train F1_SCORE: 0.0000, Valid Loss: 0.2074, Valid F1_SCORE: 0.0000,
Epoch [9] Train Loss: 0.2144, Train F1_SCORE: 0.0000, Valid Loss: 0.1883, Valid F1_SCORE: 0.0000,
Epoch [10] Train Loss: 0.1870, Train F1_SCORE: 0.0000, Valid Loss: 0.1750, Valid F1_SCORE: 0.0129,
Epoch [11] Train Lo

Some weights of the model checkpoint at yiyanghkust/finbert-tone were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
